# Ensemble Testing (Under Construction)

We have created a python module that makes the homogeneous ensemble callable.  The module is `homogeneous_ensemble.py`.

In [1]:
import homogeneous_ensemble as he

In [6]:
import numpy as np
import pandas as pd

import random
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier

abalone = pd.read_csv("abalone.csv")

training, valid = he.split_train_test(abalone, 0.2)

In [7]:
weights, predictors = he.homog_ens(training, 0, 2)
print(weights)

[0.13876194432904032, 0.16197866149369544]


In [8]:
print(predictors)

[DecisionTreeClassifier(max_depth=5), DecisionTreeClassifier(max_depth=5)]


In [12]:
X_vars = valid.iloc[:,1:-1]
X_labels = valid.iloc[:,-1]

X_vars.iloc[:5,:]

,Length,Diameter,Height,Whole_weight,Shucked_weight,Viscera_weight,Shell_weight
1009,0.530,0.400,0.125,0.6170,0.2790,0.1270,0.190
2179,0.565,0.435,0.155,0.7820,0.2715,0.1680,0.285
848,0.450,0.335,0.105,0.4470,0.2335,0.1530,0.119
1291,0.745,0.570,0.215,2.2500,1.1565,0.4460,0.558
237,0.545,0.460,0.160,0.8975,0.3410,0.1655,0.345


In [16]:
predictions = []
for p in predictors:
    predictions.append(p.predict(X_vars))

C:\Users\jcarew42\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(
C:\Users\jcarew42\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


In [12]:
sub_train.shape

(2672, 9)

In [14]:
mean_squared_error(predictors[0].predict(X_vars), X_labels)

C:\Users\jcarew42\Anaconda3\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but DecisionTreeClassifier was fitted without feature names
  warnings.warn(


6.23021582733813

In [13]:
test.shape

(668, 9)